In [1]:
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from torch.nn import LSTM
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import kneighbors_graph

# 데이터 로드
file_path = 'tidal_current_data.csv'
data = pd.read_csv(file_path)

# 결측치 처리
data['current_speed'].fillna(data['current_speed'].mean(), inplace=True)  # 유속 결측치 처리
data['current_dir'].fillna(method='ffill', inplace=True)  # 유향 결측치 처리

# 개별 정규화
scaler_dir = MinMaxScaler()  # 유향 정규화
scaler_speed = MinMaxScaler()  # 유속 정규화
data['current_dir'] = scaler_dir.fit_transform(data[['current_dir']])
data['current_speed'] = scaler_speed.fit_transform(data[['current_speed']])

# 그래프 데이터 준비
time_steps = sorted(data['time'].unique())
node_features_list = []
edge_index_list = []

for t in time_steps:
    time_data = data[data['time'] == t]
    coords = time_data[['pre_lat', 'pre_lon']].to_numpy()

    # 노드 특성
    node_features = torch.tensor(
        time_data[['current_dir', 'current_speed', 'pre_lat', 'pre_lon']].to_numpy(),
        dtype=torch.float
    )
    node_features_list.append(node_features)

    # Edge 연결
    adj_matrix = kneighbors_graph(coords, n_neighbors=5, mode='connectivity', include_self=True)
    edge_index = torch.tensor(np.array(adj_matrix.nonzero()), dtype=torch.long)
    edge_index_list.append(edge_index)

# GCN + LSTM 모델 정의
class GCNLSTM(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, lstm_hidden, out_channels):
        super(GCNLSTM, self).__init__()
        self.gcn = GCNConv(in_channels, hidden_channels)
        self.lstm = LSTM(hidden_channels, lstm_hidden, batch_first=True)
        self.fc = torch.nn.Linear(lstm_hidden, out_channels)

    def forward(self, node_features_list, edge_index_list):
        gcn_outputs = []
        for t in range(len(node_features_list)):
            gcn_out = F.relu(self.gcn(node_features_list[t], edge_index_list[t]))
            gcn_outputs.append(gcn_out)  # 각 시간 단계에서 노드별 출력
        gcn_outputs = torch.stack(gcn_outputs, dim=1)  # [노드 수, 시간 스텝, GCN 출력 크기]
        lstm_out, _ = self.lstm(gcn_outputs)
        return self.fc(lstm_out)

# 모델 초기화
model = GCNLSTM(in_channels=node_features_list[0].shape[1], hidden_channels=16, lstm_hidden=32, out_channels=2)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.MSELoss()

# 학습 루프
model.train()
epochs = 100
for epoch in range(epochs):
    optimizer.zero_grad()
    
    # 모델 출력
    pred = model(node_features_list, edge_index_list)  # [노드 수, 시간 스텝, 출력 채널 수]
    
    # 실제값 크기 맞추기
    labels = []
    for t in time_steps:
        time_data = data[data['time'] == t]
        labels.append(torch.tensor(
            time_data[['current_dir', 'current_speed']].to_numpy(), dtype=torch.float
        ))
    labels = torch.stack(labels, dim=1)  # [노드 수, 시간 스텝, 출력 채널 수]
    
    # 손실 계산
    loss = criterion(pred, labels)
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item()}")

# 예측 (마지막 시간 스텝 기준)
model.eval()
with torch.no_grad():
    last_step = -1  # 마지막 시간 스텝
    pred = model(node_features_list, edge_index_list)[:, last_step, :].numpy()  # 마지막 시간 스텝 예측값

    # 유향과 유속 각각 역정규화
    pred_dir_original = scaler_dir.inverse_transform(pred[:, 0].reshape(-1, 1)).flatten()  # 유향 복원
    pred_speed_original = scaler_speed.inverse_transform(pred[:, 1].reshape(-1, 1)).flatten()  # 유속 복원

# 결과를 Excel로 저장
last_time_data = data[data['time'] == time_steps[last_step]]
# output_data = last_time_data[['node_id', 'pre_lat', 'pre_lon']].copy()
output_data = last_time_data[['pre_lat', 'pre_lon']].copy()
output_data['유향'] = pred_dir_original  # 유향
output_data['유속'] = pred_speed_original  # 유속

output_data.to_excel("prediction_results_nodewise.xlsx", index=False)
print("결과가 prediction_results_nodewise.xlsx에 저장되었습니다.")


C:\Users\TQ\AppData\Local\Temp\ipykernel_372\1757292247.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['current_speed'].fillna(data['current_speed'].mean(), inplace=True)  # 유속 결측치 처리
C:\Users\TQ\AppData\Local\Temp\ipykernel_372\1757292247.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values alway

Epoch 0, Loss: 0.06290409713983536
Epoch 10, Loss: 0.0560070164501667
Epoch 20, Loss: 0.05380556732416153
Epoch 30, Loss: 0.05290112644433975
Epoch 40, Loss: 0.052507009357213974
Epoch 50, Loss: 0.052182625979185104
Epoch 60, Loss: 0.051687587052583694
Epoch 70, Loss: 0.051441967487335205
Epoch 80, Loss: 0.051074665039777756
Epoch 90, Loss: 0.0506141297519207
결과가 prediction_results_nodewise.xlsx에 저장되었습니다.


In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from torch.nn import LSTM
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import kneighbors_graph

# 데이터 로드
file_path = 'tidal_current_data.csv'
data = pd.read_csv(file_path)

# 결측치 처리
data['current_speed'].fillna(data['current_speed'].mean(), inplace=True)  # 유속 결측치 처리
data['current_dir'].fillna(method='ffill', inplace=True)  # 유향 결측치 처리

# 정규화
scaler = MinMaxScaler()
data[['current_dir', 'current_speed', 'pre_lat', 'pre_lon']] = scaler.fit_transform(
    data[['current_dir', 'current_speed', 'pre_lat', 'pre_lon']]
)

# 시간별로 데이터 분리 및 그래프 생성
time_steps = sorted(data['time'].unique())
node_features_list = []
edge_index_list = []

# 노드별 특성과 연결 정보 생성
for t in time_steps:
    time_data = data[data['time'] == t]
    coords = time_data[['pre_lat', 'pre_lon']].to_numpy()

    # 노드 특성
    node_features = torch.tensor(
        time_data[['current_dir', 'current_speed', 'pre_lat', 'pre_lon']].to_numpy(),
        dtype=torch.float
    )
    node_features_list.append(node_features)

    # Edge 연결
    adj_matrix = kneighbors_graph(coords, n_neighbors=5, mode='connectivity', include_self=True)
    edge_index = torch.tensor(np.array(adj_matrix.nonzero()), dtype=torch.long)
    edge_index_list.append(edge_index)

# GCN + LSTM 모델 정의
class GCNLSTM(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, lstm_hidden, out_channels):
        super(GCNLSTM, self).__init__()
        self.gcn = GCNConv(in_channels, hidden_channels)
        self.lstm = LSTM(hidden_channels, lstm_hidden, batch_first=True)
        self.fc = torch.nn.Linear(lstm_hidden, out_channels)

    def forward(self, node_features_list, edge_index_list):
        gcn_outputs = []
        for t in range(len(node_features_list)):
            gcn_out = F.relu(self.gcn(node_features_list[t], edge_index_list[t]))
            gcn_outputs.append(gcn_out)  # 각 시간 단계에서 노드별 출력
        gcn_outputs = torch.stack(gcn_outputs, dim=1)  # [노드 수, 시간 스텝, GCN 출력 크기]
        lstm_out, _ = self.lstm(gcn_outputs)
        return self.fc(lstm_out)

# 모델 초기화
model = GCNLSTM(in_channels=node_features_list[0].shape[1], hidden_channels=16, lstm_hidden=32, out_channels=2)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.MSELoss()

# 학습 루프
model.train()
epochs = 100
for epoch in range(epochs):
    optimizer.zero_grad()
    
    # 모델 출력
    pred = model(node_features_list, edge_index_list)  # [노드 수, 시간 스텝, 출력 채널 수]
    
    # 실제값 크기 맞추기
    labels = []
    for t in time_steps:
        time_data = data[data['time'] == t]
        labels.append(torch.tensor(
            time_data[['current_dir', 'current_speed']].to_numpy(), dtype=torch.float
        ))
    labels = torch.stack(labels, dim=1)  # [노드 수, 시간 스텝, 출력 채널 수]
    
    # 손실 계산
    loss = criterion(pred, labels)
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item()}")

# 예측 (10분, 20분, 30분)
model.eval()
predictions = {}
time_intervals = ["10분", "20분", "30분"]
for future_step, future_time in enumerate(time_intervals, start=1):
    with torch.no_grad():
        pred = model(node_features_list, edge_index_list)  # [노드 수, 시간 스텝, 출력 채널 수]
        predictions[future_time] = pred[:, future_step - 1, :].numpy()  # 해당 시간 스텝 예측값 저장

# 결과를 Excel로 저장
output_data = pd.DataFrame()
for future_time, pred in predictions.items():
    output_data[future_time + "_유향"] = pred[:, 0]  # 유향 예측값
    output_data[future_time + "_유속"] = pred[:, 1]  # 유속 예측값

output_data.to_excel("prediction_results_nodewise.xlsx", index=False)
print("결과가 prediction_results_nodewise.xlsx에 저장되었습니다.")


In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import kneighbors_graph

# 데이터 로드
file_path = 'tidal_current_data.csv'
data = pd.read_csv(file_path)

# 결측치 처리
data['current_speed'].fillna(data['current_speed'].mean(), inplace=True)  # 유속 결측치 처리
data['current_dir'].fillna(method='ffill', inplace=True)  # 유향 결측치 처리

# 정규화
scaler = MinMaxScaler()
data[['time', 'current_dir', 'current_speed', 'pre_lat', 'pre_lon']] = scaler.fit_transform(
    data[['time', 'current_dir', 'current_speed', 'pre_lat', 'pre_lon']]
)

# 그래프 데이터 생성
nodes = data[['pre_lat', 'pre_lon']].drop_duplicates().reset_index(drop=True)  # 고유한 노드
node_map = {tuple(x): i for i, x in nodes.iterrows()}  # 노드 인덱스 매핑

# 각 시간별 그래프 데이터 생성
time_steps = sorted(data['time'].unique())
node_features_list = []
edge_index_list = []

for t in time_steps:
    time_data = data[data['time'] == t]
    
    # 노드별 피처 생성
    features = time_data[['time', 'current_dir', 'current_speed', 'pre_lat', 'pre_lon']].to_numpy()
    node_features = torch.tensor(features, dtype=torch.float)
    node_features_list.append(node_features)
    
    # Edge 연결 (k-Nearest Neighbors)
    coords = time_data[['pre_lat', 'pre_lon']].to_numpy()
    adj_matrix = kneighbors_graph(coords, n_neighbors=5, mode='connectivity', include_self=True)
    edge_index = torch.tensor(np.array(adj_matrix.nonzero()), dtype=torch.long)
    edge_index_list.append(edge_index)

# GCN 모델 정의
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x

# 모델 초기화
in_channels = node_features_list[0].shape[1]  # 입력 피처 크기
model = GCN(in_channels=in_channels, hidden_channels=16, out_channels=2)  # 유향, 유속 예측
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.MSELoss()

# 학습 데이터: 마지막 시간 스텝 기준
X_train = node_features_list[-2]  # 마지막에서 두 번째 시간 스텝
y_train = node_features_list[-1][:, 1:3]  # 마지막 시간 스텝의 유향, 유속

# 학습 루프
epochs = 100
model.train()
for epoch in range(epochs):
    optimizer.zero_grad()
    pred = model(X_train, edge_index_list[-2])  # 예측값
    loss = criterion(pred, y_train)  # 손실 계산
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item()}")

# 예측: 마지막 시간 기준 10분 뒤
model.eval()
with torch.no_grad():
    X_test = node_features_list[-1]  # 마지막 시간 스텝 피처
    edge_index = edge_index_list[-1]  # 마지막 시간 스텝 엣지
    pred = model(X_test, edge_index).numpy()  # 예측값

# 역정규화
data[['time', 'current_dir', 'current_speed', 'pre_lat', 'pre_lon']] = scaler.inverse_transform(
    data[['time', 'current_dir', 'current_speed', 'pre_lat', 'pre_lon']]
)
pred[:, 0] = scaler.inverse_transform(pred[:, 0].reshape(-1, 1)).flatten()  # 유향
pred[:, 1] = scaler.inverse_transform(pred[:, 1].reshape(-1, 1)).flatten()  # 유속

# 결과 저장
results = nodes.copy()
results['유향_예측'] = pred[:, 0]
results['유속_예측'] = pred[:, 1]
results.to_excel("prediction_results.xlsx", index=False)
print("결과가 prediction_results.xlsx에 저장되었습니다.")


In [13]:
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from scipy.spatial.distance import cdist

# 데이터 로드
file_path = 'tidal_current_data.csv'
data = pd.read_csv(file_path)

# 결측치 처리
data['current_speed'].fillna(data['current_speed'].mean(), inplace=True)
data['current_dir'].fillna(method='ffill', inplace=True)

# 시간 데이터 변환
data['time'] = pd.to_datetime(data['time'], format='%H:%M')
data['time_minutes'] = data['time'].dt.hour * 60 + data['time'].dt.minute

# 시간의 주기적 특성 변환
data['time_sin'] = np.sin(2 * np.pi * data['time_minutes'] / 1440)
data['time_cos'] = np.cos(2 * np.pi * data['time_minutes'] / 1440)

# 정규화 (개별 처리)
scaler_features = MinMaxScaler()
scaler_coords = MinMaxScaler()
data[['current_dir', 'current_speed']] = scaler_features.fit_transform(
    data[['current_dir', 'current_speed']]
)
data[['pre_lat', 'pre_lon']] = scaler_coords.fit_transform(data[['pre_lat', 'pre_lon']])

# 고유 노드 및 인덱스 매핑
nodes = data[['pre_lat', 'pre_lon']].drop_duplicates().reset_index(drop=True)
node_map = {tuple(x): i for i, x in nodes.iterrows()}

# 그래프 데이터 생성
time_steps = sorted(data['time'].unique())
node_features_list = []
edge_index_list = []

for t in time_steps:
    time_data = data[data['time'] == t]

    # 데이터 타입 강제 변환
    time_data['pre_lat'] = pd.to_numeric(time_data['pre_lat'], errors='coerce')
    time_data['pre_lon'] = pd.to_numeric(time_data['pre_lon'], errors='coerce')
    time_data['current_dir'] = pd.to_numeric(time_data['current_dir'], errors='coerce')
    time_data['current_speed'] = pd.to_numeric(time_data['current_speed'], errors='coerce')

    # 결측치 처리
    time_data[['pre_lat', 'pre_lon', 'current_dir', 'current_speed']] = (
        time_data[['pre_lat', 'pre_lon', 'current_dir', 'current_speed']].fillna(0)
    )

    # 노드 피처 생성
    features = time_data[['time_sin', 'time_cos', 'pre_lat', 'pre_lon', 'current_dir', 'current_speed']].to_numpy()
    node_features = torch.tensor(features, dtype=torch.float)
    node_features_list.append(node_features)

    # Edge 연결 (거리 + 유사성 기반)
    coords = time_data[['pre_lat', 'pre_lon']].to_numpy()
    features = time_data[['current_dir', 'current_speed']].to_numpy()

    distance_matrix = np.array(cdist(coords, coords, metric='euclidean'))
    feature_matrix = np.array(cdist(features, features, metric='euclidean'))

    alpha, beta = 0.5, 0.5
    combined_matrix = alpha * distance_matrix + beta * feature_matrix
    adj_matrix = (combined_matrix < np.percentile(combined_matrix, 10)).astype(float)

    edge_index = torch.tensor(np.array(adj_matrix.nonzero()), dtype=torch.long)
    edge_index_list.append(edge_index)

# 모델, 학습, 예측 및 저장 (이전 코드 동일)



C:\Users\TQ\AppData\Local\Temp\ipykernel_372\4034794815.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['current_speed'].fillna(data['current_speed'].mean(), inplace=True)
C:\Users\TQ\AppData\Local\Temp\ipykernel_372\4034794815.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as 

In [20]:
import pandas as pd
import numpy as np
import torch
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import NearestNeighbors

# 데이터 로드
df = pd.read_csv('tidal_current_data.csv')  # 원본 데이터는 df로 유지
df['time'] = pd.to_datetime(df['time'], format='%H:%M')  # 시간 데이터 변환

# 시간 데이터를 주기적 특성으로 변환
df['time_minutes'] = df['time'].dt.hour * 60 + df['time'].dt.minute
df['time_sin'] = np.sin(2 * np.pi * df['time_minutes'] / 1440)  # 하루 주기
df['time_cos'] = np.cos(2 * np.pi * df['time_minutes'] / 1440)

# 입력 피처와 출력 데이터 정의
features = df[['time_sin', 'time_cos', 'pre_lat', 'pre_lon', 'current_dir', 'current_speed']].values
node_features = torch.tensor(features, dtype=torch.float)

# 노드 좌표 (위도, 경도)
coordinates = df[['pre_lat', 'pre_lon']].values

# 좌표를 라디안 단위로 변환
radians = np.radians(coordinates)

# KNN 그래프 생성
knn = NearestNeighbors(n_neighbors=10, metric='haversine')  # 각 노드와 가장 가까운 10개의 노드
knn.fit(radians)
distances, indices = knn.kneighbors(radians)

# 인접 행렬 생성 (엣지 리스트로 변환)
edge_index = []
edge_weight = []
for i, neighbors in enumerate(indices):
    for j, neighbor in enumerate(neighbors):
        edge_index.append([i, neighbor])
        edge_weight.append(distances[i][j])

edge_index = np.array(edge_index).T  # PyTorch Geometric에서 사용하는 형식으로 변환
edge_weight = np.array(edge_weight)

# PyTorch 텐서로 변환
edge_index = torch.tensor(edge_index, dtype=torch.long)
edge_weight = torch.tensor(edge_weight, dtype=torch.float)

# 그래프 데이터 생성
graph_data = Data(x=node_features, edge_index=edge_index, edge_attr=edge_weight)

# GCN 모델 정의
class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr
        x = self.conv1(x, edge_index, edge_weight=edge_attr)
        x = torch.relu(x)
        x = self.conv2(x, edge_index, edge_weight=edge_attr)
        return x

# 모델 초기화
input_dim = node_features.shape[1]
hidden_dim = 16
output_dim = 2  # 유향(current_dir), 유속(current_speed)
model = GCN(input_dim, hidden_dim, output_dim)

# 학습 설정
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.MSELoss()

# 데이터 정규화 (유향과 유속)
scaler = MinMaxScaler()
target = df[['current_dir', 'current_speed']].values  # 이제 df에서 참조
target_scaled = scaler.fit_transform(target)
target_tensor = torch.tensor(target_scaled, dtype=torch.float)

# 학습
model.train()
for epoch in range(100):  # 에포크 수 조정 가능
    optimizer.zero_grad()
    pred = model(graph_data)
    loss = criterion(pred, target_tensor)
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

# 예측
model.eval()
with torch.no_grad():
    pred = model(graph_data)
    pred_original = scaler.inverse_transform(pred.numpy())  # 역정규화

# 결과 저장
result_df = pd.DataFrame(pred_original, columns=['predicted_current_dir', 'predicted_current_speed'])
result_df['node_id'] = range(len(result_df))
result_df['pre_lat'] = df['pre_lat']
result_df['pre_lon'] = df['pre_lon']
result_df.to_excel('predicted_results.xlsx', index=False)


Epoch 0, Loss: 691.427734375
Epoch 10, Loss: 77.76171112060547
Epoch 20, Loss: 33.16193389892578
Epoch 30, Loss: 21.489887237548828
Epoch 40, Loss: 7.9299702644348145
Epoch 50, Loss: 4.238471508026123
Epoch 60, Loss: 3.5203192234039307
Epoch 70, Loss: 2.307295083999634
Epoch 80, Loss: 1.9136055707931519
Epoch 90, Loss: 1.5441490411758423


In [7]:
print(data['time'].head())  # 시간 형식 확인
print(data['time'].dtype)  # 데이터 타입 출력


0    00:00
1    00:00
2    00:00
3    00:00
4    00:00
Name: time, dtype: object
object


In [8]:
# 시간 데이터를 datetime으로 변환
data['time'] = pd.to_datetime(data['time'], format='%H:%M')

# 시간을 숫자형으로 변환 (단위: 분)
data['time_minutes'] = data['time'].dt.hour * 60 + data['time'].dt.minute


In [9]:
# 주기적 특성 변환 (하루 = 1440분)
data['time_sin'] = np.sin(2 * np.pi * data['time_minutes'] / 1440)
data['time_cos'] = np.cos(2 * np.pi * data['time_minutes'] / 1440)


In [10]:
# 시간 피처 추가
features = time_data[['time_sin', 'time_cos', 'pre_lat', 'pre_lon', 'current_dir', 'current_speed']].to_numpy()
node_features = torch.tensor(features, dtype=torch.float)


KeyError: "['time_sin', 'time_cos'] not in index"